In [34]:
import pymongo
import pandas as pd
import jieba
import numpy as np
import string
from collections import Counter
from sklearn.neighbors import NearestNeighbors

In [35]:
# mongodb uri
conn_str = "mongodb://1.15.118.125:27017/"
# 客户端连接
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
collection_user = client.get_database("Lab").get_collection("Mission")

In [62]:
# 查询 title 与对应的 tags
title_with_tags = {}
for doc in collection_user.find():
    mission_tags = "missionTags"
    mission_title = doc.get("title")
    title_with_tags[mission_title] = mission_tags

# pands 格式化输出
ser_tag_tags = pd.Series(title_with_tags)
ser_tag_tags.head(len(ser_tag_tags))

学生骨干成长训练营举办暑期专题培训                 missionTags
新生教师班主任工作研讨会召开                    missionTags
2022级本科生线上新生家长会                   missionTags
2022级研究生新生入学教育活动                  missionTags
研究生班级新生见面会顺利举行                    missionTags
                                     ...     
管理学院第二届工商管理学科高端论坛系列活动: 学科建设座谈会    missionTags
新进老师培训                            missionTags
学生骨干训练营的结营仪式                      missionTags
十二月舍导交流                           missionTags
中国管理学前沿研究研讨会: 开幕式与主报告             missionTags
Length: 101, dtype: object

In [63]:
# 忽略词
eng_words = list(string.ascii_lowercase + string.ascii_uppercase)
exclude_words = ['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&','月','第二届', '第一次','日','与','华中科技大学','年度','顺利','—','电脑','暨','大学','关西','十一月','学院','来','｜', '第一','级','商学院','之','做','科','会','|','啦']
for w in eng_words:
    exclude_words.append(w)
for i in range(10):
    exclude_words.append(str(i))
print(exclude_words)

['，', '“', '”', '、', '《', '》', '。', ':', '-', '的', '?', ' ', ',', '&', '月', '第二届', '第一次', '日', '与', '华中科技大学', '年度', '顺利', '—', '电脑', '暨', '大学', '关西', '十一月', '学院', '来', '｜', '第一', '级', '商学院', '之', '做', '科', '会', '|', '啦', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [64]:
# 分割 title 的词语
total_words = []
title_with_words = {}
for title in title_with_tags.keys():
    title_copy = title
    for i in exclude_words:
        title_copy = title.replace(i, '')
    title_with_words[title] = []
    # jieba拆分方法
    words = jieba.cut(title_copy, cut_all=False)
    for word in words:
        total_words.append(word)
        title_with_words[title].append(word)

fre = Counter(total_words)
word_fre = sorted(fre.items(), key=lambda i: i[1], reverse=True)
print("总词数: " + len(word_fre).__str__())
print(word_fre)

总词数: 370
[('活动', 25), ('2022', 19), (' ', 15), ('级', 13), ('顺利', 13), ('分享', 13), ('举行', 12), ('暨', 12), ('“', 12), ('”', 12), ('班', 12), ('开展', 11), ('管理', 11), ('主题', 10), ('二十大', 10), ('召开', 9), ('的', 9), ('精神', 9), ('学习', 8), ('新生', 7), ('论坛', 7), ('学术', 7), ('会', 7), ('党', 7), ('学生', 6), ('举办', 6), ('培训', 6), ('入学', 6), ('MBA', 6), ('做', 6), ('班会', 6), ('学院', 6), ('见面会', 5), ('大会', 5), ('年', 5), ('宣讲会', 5), ('竞赛', 5), ('喻园', 5), ('大学', 5), ('党支部', 5), ('月', 5), ('知识', 5), ('2201', 5), ('骨干', 4), ('专题', 4), ('本科生', 4), ('教育', 4), ('教授', 4), ('商学院', 4), ('师生', 4), ('顺利开展', 4), ('、', 4), ('座谈会', 4), ('十一月', 4), ('啦', 4), ('党史', 4), ('德育', 4), (':', 4), ('训练营', 3), ('工作', 3), ('研讨会', 3), ('1', 3), ('党员', 3), ('舍导进', 3), ('宿舍', 3), ('有限公司', 3), ('国奖', 3), ('&', 3), ('成功', 3), ('杯', 3), ('课程', 3), ('院长', 3), ('工商', 3), ('2205', 3), ('经验', 3), ('团支部', 3), ('第一次', 3), ('本科', 3), ('党日', 3), ('与', 3), ('贯彻', 3), ('｜', 3), ('仪式', 3), ('项目', 3), ('电脑', 3), ('交流', 3), ('集体', 3), ('领学团', 3), ('

In [65]:
print(word_fre)
for tuple in word_fre:
    if tuple[1] == 1:
        total_words.remove(tuple[0])
print(total_words.__len__())

[('活动', 25), ('2022', 19), (' ', 15), ('级', 13), ('顺利', 13), ('分享', 13), ('举行', 12), ('暨', 12), ('“', 12), ('”', 12), ('班', 12), ('开展', 11), ('管理', 11), ('主题', 10), ('二十大', 10), ('召开', 9), ('的', 9), ('精神', 9), ('学习', 8), ('新生', 7), ('论坛', 7), ('学术', 7), ('会', 7), ('党', 7), ('学生', 6), ('举办', 6), ('培训', 6), ('入学', 6), ('MBA', 6), ('做', 6), ('班会', 6), ('学院', 6), ('见面会', 5), ('大会', 5), ('年', 5), ('宣讲会', 5), ('竞赛', 5), ('喻园', 5), ('大学', 5), ('党支部', 5), ('月', 5), ('知识', 5), ('2201', 5), ('骨干', 4), ('专题', 4), ('本科生', 4), ('教育', 4), ('教授', 4), ('商学院', 4), ('师生', 4), ('顺利开展', 4), ('、', 4), ('座谈会', 4), ('十一月', 4), ('啦', 4), ('党史', 4), ('德育', 4), (':', 4), ('训练营', 3), ('工作', 3), ('研讨会', 3), ('1', 3), ('党员', 3), ('舍导进', 3), ('宿舍', 3), ('有限公司', 3), ('国奖', 3), ('&', 3), ('成功', 3), ('杯', 3), ('课程', 3), ('院长', 3), ('工商', 3), ('2205', 3), ('经验', 3), ('团支部', 3), ('第一次', 3), ('本科', 3), ('党日', 3), ('与', 3), ('贯彻', 3), ('｜', 3), ('仪式', 3), ('项目', 3), ('电脑', 3), ('交流', 3), ('集体', 3), ('领学团', 3), ('导师', 3), 

In [66]:
# 构建词语数据矩阵
dist = np.zeros((len(title_with_tags), len(set(total_words))))
print(dist.shape)

(101, 154)


In [67]:
# 将数据填入矩阵
total_words_list = list(set(total_words))
title_list = list(title_with_words.keys())
print(title_list)
for i in range(len(title_list)):
    print(title_with_words[title_list[i]])
    for word in title_with_words[title_list[i]]:
        for j in range(len(total_words_list)):
            if word == total_words_list[j]:
                print(word)
                dist[i][j] += 1

['学生骨干成长训练营举办暑期专题培训', '新生教师班主任工作研讨会召开', '2022级本科生线上新生家长会', '2022级研究生新生入学教育活动', '研究生班级新生见面会顺利举行', '19级就业动员会暨年级大会', '2022年“红色领航员”党员舍导进宿舍活动', '赴湖北稳健医疗集团有限公司开展暑期实践调研活动', '2022年暑期社会实践分享交流会', '2022年新生安全心理主题教育', '2022国奖国励宣讲会顺利举办', '简历指导&1对1简历修改活动成功举办', '2022年度秋季学期教职工大会举行', '新生杯足球赛', '华中科技大学2022级MBA开学典礼暨入学导航', '华中大2022年秋季学期教师教学竞赛赛前培训', '喻园管理论坛 | 香港城市大学教授陈旸旸做学术分享', '会计专业课程组研讨会召开', '日本关西大学商学院院长矢田勝俊教授做学术分享', '日本关西大学商学院副教授李振做学术分享', '2022级MBA入学活动—龙舟赛', '2022级MBA迎新晚会活动精彩上演', '2022级MBA入学活动—奔跑吧MBA', '2022级MBA入学活动“商工融合.创新创业”讲座顺利举行', '2022年度国家奖学金评选答辩会成功举行', '2022级MPAcc第一党支部召开九月组织生活会', '传统节日思想文化主题教育之重阳节敬老行', '师生共看庆祝华中科技大学建校70周年大会', '2022级本科生入学教育大会暨年级大会顺利召开', '2022级本科生班主任见面会暨专业导航会', 'MPAcc2204班党团共建活动顺利开展', '工商2205和2206班联合开展朋辈经验分享主题班会', '新生杯足球赛晋级八强', '工商2202班、2204班团支部十月联合主题团日活动顺利开展', '新闻宣传中心第二次例会暨第一次培训顺利举行', '工商管理专业师生座谈会顺利举行', '本科第五党支部开展10月主题党日活动', '2022级MPAcc第一党支部开展主题党日活动', '学硕管理科学与工程党支部学习贯彻党的二十大精神', '国奖专访', '喻园管理论坛｜万得信息技术股份有限公司客户经理朱同伟做Wind金融终端产品培训', '课堂进企业 | 劲牌有限公司：探索数字化营销之路', '开展9月党员舍导进宿舍活动', '我院召开国际认证工

In [68]:
# 使用 KNN 算法计算样本相互距离
calcu_nbr = NearestNeighbors(n_neighbors=12, algorithm='ball_tree').fit(dist)
distances,indices = calcu_nbr.kneighbors(dist)

In [75]:
np.set_printoptions(precision=3)
np.set_printoptions(threshold = 10000)
print(distances)

[[0.    2.646 2.646 2.646 2.828 2.828 2.828 3.    3.    3.    3.162 3.162]
 [0.    2.449 2.449 2.449 2.449 2.646 2.646 2.646 2.646 2.646 2.646 2.646]
 [0.    2.    2.    2.    2.    2.236 2.236 2.236 2.236 2.236 2.236 2.449]
 [0.    2.    2.    2.    2.236 2.449 2.449 2.449 2.646 2.646 2.646 2.646]
 [0.    1.732 1.732 1.732 1.732 2.    2.    2.    2.    2.    2.236 2.236]
 [0.    2.236 2.236 2.449 2.449 2.449 2.449 2.449 2.646 2.646 2.646 2.646]
 [0.    2.449 2.449 2.646 2.646 2.828 2.828 2.828 2.828 3.    3.    3.   ]
 [0.    2.    2.236 2.236 2.236 2.449 2.449 2.449 2.449 2.646 2.646 2.646]
 [0.    2.236 2.236 2.449 2.449 2.449 2.449 2.646 2.646 2.646 2.646 2.646]
 [0.    2.236 2.449 2.449 2.449 2.449 2.646 2.646 2.646 2.646 2.646 2.646]
 [0.    2.    2.    2.236 2.236 2.236 2.449 2.449 2.449 2.449 2.646 2.646]
 [0.    3.317 3.464 3.464 3.606 3.606 3.606 3.606 3.606 3.606 3.742 3.742]
 [0.    2.    2.449 2.449 2.646 2.646 2.646 2.646 2.646 2.646 2.828 2.828]
 [0.    0.    1.414 1.732

In [78]:
# 根据矩阵进行寻找
def find_similar_mission(title_str,num):
    similar_mission_list = []
    my_index = title_list.index(title_str)
    print("为 " + title_str+" 寻找相似任务")
    pointer = 0
    count = 0
    while count < num:
        if distances[my_index][pointer]>2:
            break
        if indices[my_index][pointer] == my_index:
            pointer += 1
            continue
        else:
            similar_mission_list.append(title_list[indices[my_index][pointer]])
            count += 1
            pointer += 1
    for mis in similar_mission_list:
        print(" -> "+mis)
    return similar_mission_list

In [79]:
get_num = 2
for title in title_list:
    find_similar_mission(title,get_num)


为 学生骨干成长训练营举办暑期专题培训 寻找相似任务
为 新生教师班主任工作研讨会召开 寻找相似任务
为 2022级本科生线上新生家长会 寻找相似任务
 -> 2022级MBA迎新晚会活动精彩上演
 -> 校运会跟拍
为 2022级研究生新生入学教育活动 寻找相似任务
 -> 2022级本科生线上新生家长会
 -> 2022级MBA迎新晚会活动精彩上演
为 研究生班级新生见面会顺利举行 寻找相似任务
 -> 财政金融管理系师生座谈会顺利举行
 -> 保研逐梦正当时
为 19级就业动员会暨年级大会 寻找相似任务
为 2022年“红色领航员”党员舍导进宿舍活动 寻找相似任务
为 赴湖北稳健医疗集团有限公司开展暑期实践调研活动 寻找相似任务
 -> 热缩片DIY活动
为 2022年暑期社会实践分享交流会 寻找相似任务
为 2022年新生安全心理主题教育 寻找相似任务
为 2022国奖国励宣讲会顺利举办 寻找相似任务
 -> 选调政策宣讲会顺利举办
 -> 国奖专访
为 简历指导&1对1简历修改活动成功举办 寻找相似任务
为 2022年度秋季学期教职工大会举行 寻找相似任务
 -> 2022年度国家奖学金评选答辩会成功举行
为 新生杯足球赛 寻找相似任务
 -> 新生杯足球赛晋级八强
 -> 华工杯乒乓球赛半决赛
为 华中科技大学2022级MBA开学典礼暨入学导航 寻找相似任务
为 华中大2022年秋季学期教师教学竞赛赛前培训 寻找相似任务
为 喻园管理论坛 | 香港城市大学教授陈旸旸做学术分享 寻找相似任务
为 会计专业课程组研讨会召开 寻找相似任务
 -> 会计专业课程组本科教学汇报
为 日本关西大学商学院院长矢田勝俊教授做学术分享 寻找相似任务
 -> 日本关西大学商学院副教授李振做学术分享
为 日本关西大学商学院副教授李振做学术分享 寻找相似任务
 -> 日本关西大学商学院院长矢田勝俊教授做学术分享
为 2022级MBA入学活动—龙舟赛 寻找相似任务
 -> 2022级MBA入学活动—奔跑吧MBA
 -> 2022级MBA迎新晚会活动精彩上演
为 2022级MBA迎新晚会活动精彩上演 寻找相似任务
 -> 2022级MBA入学活动—龙舟赛
 -> 2022级MBA入学活动—奔跑吧MBA
为 2022级MBA入学活动—奔跑吧MBA 寻找相似任务
 -> 2022级MB

In [71]:
print(title_list)

['学生骨干成长训练营举办暑期专题培训', '新生教师班主任工作研讨会召开', '2022级本科生线上新生家长会', '2022级研究生新生入学教育活动', '研究生班级新生见面会顺利举行', '19级就业动员会暨年级大会', '2022年“红色领航员”党员舍导进宿舍活动', '赴湖北稳健医疗集团有限公司开展暑期实践调研活动', '2022年暑期社会实践分享交流会', '2022年新生安全心理主题教育', '2022国奖国励宣讲会顺利举办', '简历指导&1对1简历修改活动成功举办', '2022年度秋季学期教职工大会举行', '新生杯足球赛', '华中科技大学2022级MBA开学典礼暨入学导航', '华中大2022年秋季学期教师教学竞赛赛前培训', '喻园管理论坛 | 香港城市大学教授陈旸旸做学术分享', '会计专业课程组研讨会召开', '日本关西大学商学院院长矢田勝俊教授做学术分享', '日本关西大学商学院副教授李振做学术分享', '2022级MBA入学活动—龙舟赛', '2022级MBA迎新晚会活动精彩上演', '2022级MBA入学活动—奔跑吧MBA', '2022级MBA入学活动“商工融合.创新创业”讲座顺利举行', '2022年度国家奖学金评选答辩会成功举行', '2022级MPAcc第一党支部召开九月组织生活会', '传统节日思想文化主题教育之重阳节敬老行', '师生共看庆祝华中科技大学建校70周年大会', '2022级本科生入学教育大会暨年级大会顺利召开', '2022级本科生班主任见面会暨专业导航会', 'MPAcc2204班党团共建活动顺利开展', '工商2205和2206班联合开展朋辈经验分享主题班会', '新生杯足球赛晋级八强', '工商2202班、2204班团支部十月联合主题团日活动顺利开展', '新闻宣传中心第二次例会暨第一次培训顺利举行', '工商管理专业师生座谈会顺利举行', '本科第五党支部开展10月主题党日活动', '2022级MPAcc第一党支部开展主题党日活动', '学硕管理科学与工程党支部学习贯彻党的二十大精神', '国奖专访', '喻园管理论坛｜万得信息技术股份有限公司客户经理朱同伟做Wind金融终端产品培训', '课堂进企业 | 劲牌有限公司：探索数字化营销之路', '开展9月党员舍导进宿舍活动', '我院召开国际认证工